This example illustrates how to solve typical side-by-side task with Crowdom.

In our example, we ask workers which transcript is more suitable for the given audio.

You may want to first study [image classification](../image_classification/image_classification.ipynb) example because it contains more detailed comments of overall process.

## Setup


In [ ]:
%pip install crowdom
%pip install pyyaml
%pip install markdown2

In [ ]:
from datetime import timedelta
import logging.config
import pandas as pd

import markdown2
import yaml

from crowdom import base, datasource, client, objects, pricing, params as labeling_params

In [2]:
with open('logging.yaml') as f:
    logging.config.dictConfig(yaml.full_load(f.read()))

In [3]:
from IPython.display import clear_output, display

In [10]:
toloka_client = client.create_toloka_client(token=os.getenv('TOLOKA_TOKEN') or input('Enter your token: '))
clear_output()

## Task definition


In [29]:
lang = 'EN'

In [30]:
instruction = {}
for worker_lang in ['EN', 'RU']:
    with open(f'instruction_{worker_lang}.md') as f:
        instruction[worker_lang] = markdown2.markdown(f.read())

In [31]:
task_spec = base.TaskSpec(
    id='audio-transcript-sbs',
    function=base.SbSFunction(inputs=(objects.Text,), hints=(objects.Audio,)),
    name={
        'EN': 'Audio transcripts comparison', 
        'RU': 'Сравнение расшифровок аудиозаписей',
    },
    description={
        'EN': 'From the two transcripts, choose the more suitable for given audio recording',
        'RU': 'Из двух расшифровок выберите более подходящую аудиозаписи',
    },
    instruction=instruction,
)

In [32]:
task_spec_en = client.PreparedTaskSpec(task_spec, lang)

In [33]:
client.define_task(task_spec_en, toloka_client)

In [16]:
task_duration_hint = timedelta(seconds=10)  # audios are about 1-5 seconds each

## Data preparation


In [34]:
input_objects = datasource.read_tasks('tasks.json', task_spec_en.task_mapping)
control_objects = datasource.read_tasks('control_tasks.json', task_spec_en.task_mapping, has_solutions=True)

In [35]:
client.TaskPreview(input_objects[0], task_spec=task_spec_en).display_link()

## Launch configuration


In [ ]:
params_form = labeling_params.get_interface(task_spec_en, task_duration_hint, toloka_client)

In [20]:
params = params_form.get_params()

## Launch


In [21]:
raw_results, worker_weights = client.launch_sbs(
    task_spec_en,
    params,
    input_objects,
    control_objects,
    toloka_client,
    interactive=True,
)

clear formula, which does not account edge cases like min commission and incomplete assignments


<IPython.core.display.Math object>


more precise formula, which accounts more edge cases


<IPython.core.display.Math object>

run classification of 5 objects for 0.02$? [Y/n]  Y


Output()

2022-10-31 12:44:52,918 - crowdom.client.launch:_launch:193 - INFO: - classification has started


In [22]:
results = client.ClassificationResults(input_objects, raw_results, task_spec_en, worker_weights)

## Results study


In [28]:
with pd.option_context('max_colwidth', 100):
    display(results.predict())

,audio_hint,text_a,text_b,result,confidence,overlap
0,https://tlk.s3.yandex.net/ext_dataset/noisy_speech/noisy_tested_wav/p257_026.wav,is was accurate,is this accurate,b,1.0,1
1,https://tlk.s3.yandex.net/ext_dataset/noisy_speech/noisy_tested_wav/p232_300.wav,what has altered,has altered,a,1.0,1
2,https://tlk.s3.yandex.net/ext_dataset/noisy_speech/noisy_tested_wav/p257_120.wav,does this,does this mean,b,1.0,1
3,https://tlk.s3.yandex.net/ext_dataset/noisy_speech/noisy_tested_wav/p232_044.wav,we will push,we will pull,a,1.0,1
4,https://tlk.s3.yandex.net/ext_dataset/noisy_speech/noisy_tested_wav/p232_405.wav,we have not received a letter from the danish,we have not yet received a letter from the irish,b,1.0,1
